# California Housing Price Prediction

This notebook demonstrates a machine learning pipeline to predict median housing prices in California, using the California Housing dataset from scikit-learn. The process involves data loading, exploratory data analysis (EDA), data preprocessing (outlier handling, feature engineering, feature scaling), and training various regression models to find the best performing one.

## Table of Contents
1.  [Setup and Data Loading](#setup-and-data-loading)
2.  [Exploratory Data Analysis (EDA)](#exploratory-data-analysis-eda)
3.  [Data Preprocessing](#data-preprocessing)
4.  [Model Training and Evaluation](#model-training-and-evaluation)
5.  [Conclusion and Visualization](#conclusion-and-visualization)

## 1. Setup and Data Loading

We begin by importing all necessary libraries and loading the California Housing dataset. The dataset includes features such as median income, house age, number of rooms, and geographical coordinates, with the target variable being the median house value.

In [248]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [249]:
# Get the data from sklearn.datasets library
california_housing = fetch_california_housing()

# Create a DataFrame from the dataset features
df = pd.DataFrame.from_records(california_housing.data, columns=california_housing.feature_names)

# Assign the target variable (Median House Value)
y = california_housing.target
df[california_housing.target_names[0]] = california_housing.target

print("DataFrame Head:")
print(df.head())
print("\nDataFrame Info:")
df.info()
print("\nDataFrame Description:")
print(df.describe())

In [250]:
# Splitting the dataset into training and testing data
# We'll use 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(df.drop('MedHouseVal', axis=1), df['MedHouseVal'], test_size=0.2, random_state=42)

# Create DataFrames from the split data for easier manipulation later
df_train = pd.DataFrame(data=X_train, columns=california_housing.feature_names)
df_test = pd.DataFrame(data=X_test, columns=california_housing.feature_names)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (16512, 8)
Shape of X_test: (4128, 8)
Shape of y_train: (16512,)
Shape of y_test: (4128,)


## 2. Exploratory Data Analysis (EDA)

In this section, we'll explore the data distribution and relationships between features. This helps us understand the dataset's characteristics and identify potential issues like outliers.

### Data Distribution
Let's visualize the distribution of all features in the training data using histograms. This provides a quick overview of their ranges and skewness.

In [252]:
df_train.hist(bins=30, figsize=(15, 10))
plt.suptitle('Distribution of Features in Training Data', y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

array([[<Axes: title={'center': 'MedInc'}>,
        <Axes: title={'center': 'HouseAge'}>,
        <Axes: title={'center': 'AveRooms'}>],
       [<Axes: title={'center': 'AveBedrms'}>,
        <Axes: title={'center': 'Population'}>,
        <Axes: title={'center': 'AveOccup'}>],
       [<Axes: title={'center': 'Latitude'}>,
        <Axes: title={'center': 'Longitude'}>, <Axes: >]], dtype=object)

### Geospatial Distribution of House Values
Visualizing the median house value across geographical coordinates (Latitude and Longitude) can reveal spatial patterns.

In [253]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x="Longitude", y="Latitude", hue='MedHouseVal', alpha=0.5, palette='viridis', s=20)
plt.title('Median House Value by Location in California', fontsize=16)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)
plt.show()

<Axes: xlabel='Longitude', ylabel='Latitude'>

### Correlation Analysis
Checking the correlation matrix of the training data. Some features exhibit a negative correlation, meaning they are inversely proportional. For instance, 'HouseAge' and 'MedInc' have a correlation value of -0.12, indicating that areas with higher median incomes tend to have newer houses.

In [254]:
print(df_train.corr())

plt.figure(figsize=(10, 8))
sns.heatmap(df_train.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Training Data', fontsize=16)
plt.show()

              MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  \
MedInc      1.000000 -0.121711  0.336013  -0.072550    0.004122  0.022061   
HouseAge   -0.121711  1.000000 -0.160892  -0.087983   -0.292283  0.016245   
AveRooms    0.336013 -0.160892  1.000000   0.836163   -0.073733 -0.004922   
AveBedrms  -0.072550 -0.087983  0.836163   1.000000   -0.071267 -0.006035   
Population  0.004122 -0.292283 -0.073733  -0.071267    1.000000  0.072330   
AveOccup    0.022061  0.016245 -0.004922  -0.006035    0.072330  1.000000   
Latitude   -0.076571  0.005296  0.110695   0.073161   -0.102499  0.005837   
Longitude  -0.017040 -0.101818 -0.029339   0.016647    0.094276 -0.000598   

            Latitude  Longitude  
MedInc     -0.076571  -0.017040  
HouseAge    0.005296  -0.101818  
AveRooms    0.110695  -0.029339  
AveBedrms   0.073161   0.016647  
Population -0.102499   0.094276  
AveOccup    0.005837  -0.000598  
Latitude    1.000000  -0.924485  
Longitude  -0.924485   1.000000  


<Axes: >

## 3. Data Preprocessing

This section focuses on preparing the data for model training. This includes handling outliers, engineering new features, and scaling existing features.

### Outlier Handling
We'll check for outliers in the training data using descriptive statistics. Linear regression models are highly sensitive to outliers, which can degrade their performance.

To mitigate this, we'll implement a clipping strategy. We will use the 95th percentile as the upper limit and the 5th percentile as the lower limit. Typical percentile values used for clipping include 99-1, 95-5, and 90-10.

In [255]:
df_train.describe()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,3.880754,28.608285,5.435235,1.096685,1426.453004,3.096961,35.643149,-119.582290
std,1.904294,12.602499,2.387375,0.433215,1137.056380,11.578744,2.136665,2.005654
min,0.499900,1.000000,0.888889,0.333333,3.000000,0.692308,32.550000,-124.350000
25%,2.566700,18.000000,4.452055,1.006508,789.000000,2.428799,33.930000,-121.810000
50%,3.545800,29.000000,5.235874,1.049286,1167.000000,2.817240,34.260000,-118.510000
75%,4.773175,37.000000,6.061037,1.100348,1726.000000,3.280000,37.720000,-118.010000
max,15.000100,52.000000,141.909091,25.636364,35682.000000,1243.333333,41.950000,-114.310000


In [256]:
def clip_outlier(df_input, features, lower_quantile=0.05, upper_quantile=0.95):
    """
    Clips outliers in specified DataFrame features using quantile-based limits.

    Args:
        df_input (pd.DataFrame): The input DataFrame.
        features (list): A list of column names to clip outliers from.
        lower_quantile (float): The lower quantile to use as the lower limit (default: 0.05).
        upper_quantile (float): The upper quantile to use as the upper limit (default: 0.95).

    Returns:
        pd.DataFrame: A new DataFrame with clipped features.
    """
    df_output = df_input.copy()
    for feature in features:
        upper_limit = df_output[feature].quantile(upper_quantile)
        lower_limit = df_output[feature].quantile(lower_quantile)
        df_output[f'{feature}_capped_quantile'] = np.where(
            df_output[feature] > upper_limit,
            upper_limit,
            np.where(
                df_output[feature] < lower_limit,
                lower_limit,
                df_output[feature]
            )
        )
    return df_output

In [257]:
# Define features to clip
clip_features = ['Population', 'MedInc', 'AveRooms', 'AveBedrms', 'AveOccup']

# Apply clipping to both training and testing DataFrames
df_train = clip_outlier(df_train, clip_features)
df_test = clip_outlier(df_test, clip_features)

print("DataFrame Description After Outlier Clipping (Training Data):")
print(df_train.describe())

DataFrame Description After Outlier Clipping (Training Data):


### Feature Engineering
We'll engineer two more features to potentially improve model performance:
1.  **`AveBedrms_AveRooms_ratio`**: The ratio of the average number of bedrooms to the average number of rooms in a house. This can indicate the proportion of bedrooms to other living spaces.
2.  **`AveRooms_AveOccup_ratio`**: The ratio of the average number of rooms to the average occupancy (number of people per household). This can be interpreted as "rooms per person," offering insights into living space per individual.

In [258]:
def add_engineered_features(df_input):
    """
    Adds engineered features to the DataFrame.
    Assumes capped_quantile features already exist.
    """
    df_output = df_input.copy()
    df_output['AveBedrms_AveRooms_ratio'] = df_output['AveBedrms_capped_quantile'] / df_output['AveRooms_capped_quantile']
    df_output['AveRooms_AveOccup_ratio'] = df_output['AveRooms_capped_quantile'] / df_output['AveOccup_capped_quantile']
    return df_output

# Apply feature engineering
df_train = add_engineered_features(df_train)
df_test = add_engineered_features(df_test)

# Check correlation of new features with the target
print("Correlation of AveBedrms_AveRooms_ratio with y_train:")
print(np.corrcoef(df_train['AveBedrms_AveRooms_ratio'], y_train))

print("Correlation of AveRooms_AveOccup_ratio with y_train:")
print(np.corrcoef(df_train['AveRooms_AveOccup_ratio'], y_train))

print("\nUpdated Training DataFrame Columns:")
print(df_train.columns)

Correlation of AveBedrms_AveRooms_ratio with y_train:
[[ 1.         -0.25741882]
 [-0.25741882  1.        ]]
Correlation of AveRooms_AveOccup_ratio with y_train:
[[ 1.         -0.03115971]
 [-0.03115971  1.        ]]


### Feature Selection and Scaling
We select the relevant features, including our newly engineered ones and the capped original features, to form our final `X_train` and `X_test` datasets.

Before applying regression models, especially those sensitive to feature magnitudes like Linear Regression, it's crucial to perform feature scaling. If features have vastly different scales, the model might incorrectly attribute more importance to features with larger values, leading to suboptimal training and overfitting. `StandardScaler` transforms our data such that it has a mean of 0 and a standard deviation of 1.

It's worth noting that tree-based models like Decision Trees are generally insensitive to feature scaling because they make decisions based on impurity or entropy by splitting data based on feature values, not their magnitude.

In [259]:
# Define the list of features to be used in the models. These include capped and engineered features.
features = ['MedInc_capped_quantile', 'Latitude', 'Longitude', 
            'AveBedrms_AveRooms_ratio', 'AveRooms_AveOccup_ratio', 
            'HouseAge', 'Population_capped_quantile']

X_train_processed = df_train[features].values
X_test_processed = df_test[features].values

# Initialize StandardScaler
scaler = StandardScaler()

# Fit on training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train_processed)
X_test_scaled = scaler.transform(X_test_processed)

print(scaler)

StandardScaler()

## 4. Model Training and Evaluation

We will train and evaluate several regression models to predict housing prices. For each model, we'll report Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared ($R^2$). A higher R-squared value indicates a better fit of the model to the data.

In [260]:
def evaluate_model(model, X_train, y_train, X_test, y_test, param_grid=None):
    """
    Trains and evaluates a given regression model. Supports GridSearchCV for hyperparameter tuning.

    Args:
        model (estimator): The scikit-learn regressor model to train.
        X_train (array-like): Scaled training features.
        y_train (array-like): Training target.
        X_test (array-like): Scaled testing features.
        y_test (array-like): Testing target.
        param_grid (dict, optional): Dictionary of hyperparameters to tune with GridSearchCV. Defaults to None.

    Returns:
        tuple: (best_estimator, predictions, mae, mse, r2)
    """
    if param_grid:
        grid_search = GridSearchCV(model, param_grid, cv=3, n_jobs=-1, verbose=0)
        grid_search.fit(X_train, y_train)
        best_estimator = grid_search.best_estimator_
        print(f"Best parameters for {model.__class__.__name__}: {grid_search.best_params_}")
    else:
        best_estimator = model
        best_estimator.fit(X_train, y_train)
    
    predictions = best_estimator.predict(X_test)
    
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    return best_estimator, predictions, mae, mse, r2

### a. Linear Regression

In [261]:
lr_model = LinearRegression(fit_intercept=True)
lr_best_estimator, lr_predictions, lr_mae, lr_mse, lr_r2 = evaluate_model(lr_model, X_train_scaled, y_train, X_test_scaled, y_test)

print("Linear Regression Results:")
print(f"Mean Absolute Error (MAE): {lr_mae}")
print(f"Mean Squared Error (MSE): {lr_mse}")
print(f"R-squared (R2): {lr_r2}")

Linear Regression Results:
Mean Absolute Error (MAE): 0.4897401032906702
Mean Squared Error (MSE): 0.4459624822936078
R-squared (R2): 0.6596768721073093


### b. Decision Tree Regressor
We use `GridSearchCV` to find the optimal hyperparameters for our Decision Tree Regressor. Note that Decision Trees are less sensitive to feature scaling, but scaling is still applied for consistency in the pipeline.

In [262]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_param_grid = {
    "max_depth": [3, 5, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 3, 4, 5]
}
dt_best_estimator, dt_predictions, dt_mae, dt_mse, dt_r2 = evaluate_model(dt_model, X_train_scaled, y_train, X_test_scaled, y_test, dt_param_grid)

print("Decision Tree Regressor Results:")
print(f"Mean Absolute Error (MAE): {dt_mae}")
print(f"Mean Squared Error (MSE): {dt_mse}")
print(f"R-squared (R2): {dt_r2}")

Best parameters for DecisionTreeRegressor: {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}
Decision Tree Regressor Results:
Mean Absolute Error (MAE): 0.43189922182861074
Mean Squared Error (MSE): 0.41212809475113166
R-squared (R2): 0.6854965880160752


### c. Random Forest Regressor
Random Forests, an ensemble of Decision Trees, often provide improved accuracy and robustness. We also use `GridSearchCV` for hyperparameter tuning here.

In [263]:
rf_model = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=100) # n_jobs=-1 uses all available CPU cores
rf_param_grid = {
    "max_depth": [3, 5, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5]
}
rf_best_estimator, rf_predictions, rf_mae, rf_mse, rf_r2 = evaluate_model(rf_model, X_train_scaled, y_train, X_test_scaled, y_test, rf_param_grid)

print("Random Forest Regressor Results:")
print(f"Mean Absolute Error (MAE): {rf_mae}")
print(f"Mean Squared Error (MSE): {rf_mse}")
print(f"R-squared (R2): {rf_r2}")

Best parameters for RandomForestRegressor: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2}
Random Forest Regressor Results:
Mean Absolute Error (MAE): 0.38452772996033857
Mean Squared Error (MSE): 0.32273071015358695
R-squared (R2): 0.7537175679406422


### d. XGBoost Regressor
XGBoost is a powerful gradient boosting framework known for its high performance in many machine learning tasks. It typically performs well with default parameters but can be further optimized.

In [264]:
xgb_model = XGBRegressor(random_state=42) # Using default parameters, but could be tuned with GridSearchCV
xgb_best_estimator, xgb_predictions, xgb_mae, xgb_mse, xgb_r2 = evaluate_model(xgb_model, X_train_scaled, y_train, X_test_scaled, y_test)

print("XGBoost Regressor Results:")
print(f"Mean Absolute Error (MAE): {xgb_mae}")
print(f"Mean Squared Error (MSE): {xgb_mse}")
print(f"R-squared (R2): {xgb_r2}")

XGBoost Regressor Results:
Mean Absolute Error (MAE): 0.307848461431221
Mean Squared Error (MSE): 0.2180799337088517
R-squared (R2): 0.8335787244058696


## 5. Conclusion and Visualization

The XGBoost Regressor delivered the best performance in this analysis, with an impressive R-squared ($R^2$) score of 0.83.

Finally, we visualize the predictions from our best model (XGBoost) against the actual values. A scatter plot where points align closely to a 45-degree line indicates good model performance.

In [265]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, xgb_predictions, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Values', fontsize=12)
plt.ylabel('Predicted Values', fontsize=12)
plt.title('XGBoost: Actual vs. Predicted Housing Values', fontsize=16)
plt.grid(True)
plt.show()

The scatter plot shows that the predicted values from the XGBoost model align well with the actual values, indicating a strong predictive capability. This refactored notebook provides a clean, well-organized, and reproducible pipeline for housing price prediction.